# Prediction Model

In this notebook, we construct predictive models based on the dataset prepared in the [preceding notebook](./1--analysis.ipynb).

To evaluate the quality of the models, we select the true positive rate (i.e. recall for the presence of diabetes) as our metric, as an initial educated guess: it shall be deemed more serious to misclassify individuals afflicted with the disease as healthy than to ascribe diabetes to healthy individuals.

Naturally, this choice might require refinement should more information (e.g. regarding the severity of side effects from prescribed drugs) be available.

## Setup

In this notebook, the following modules are requisite:

In [ ]:
import os

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.tree import DecisionTreeClassifier

from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline

import matplotlib.pyplot as plt
import seaborn as sns

from ipynb_utils import CFG

Let us encapsulate certain procedures to be applied in the error analysis of each model:

In [ ]:
outcome_palette = {
    "TP": "green",
    "TN": "blue",
    "FP": "red",
    "FN": "orange",
}


def classify_outcome(row):
    if row["diagnosis"] == 1:
        if row["prognosis"] == 1:
            return "TP" 
        else:
            return "FN"
    else:
        if row["prognosis"] == 1:
            return "FP"
        else:
            return "TN"

The following code cell gathers the environment variables to be used explicitly.

In [ ]:
RSEED = CFG["RSEED"]

DATA_DIR = CFG["DATA_DIR"]

DF_PKL_PATH_SRC = os.path.join(DATA_DIR, "df_processed.pkl")

Let us reload the processed data from preceding exploratory data analysis.

In [ ]:
df = pd.read_pickle(DF_PKL_PATH_SRC)

Prior to the imputation of implausible values in the data frame, a pseudo-feature named `"is_test"` was already introduced to implement a train-test split. Naturally, utilising any data division risks other than this risks information leakage; accordingly, we shall adhere to this split.

In [ ]:
is_test = df["is_test"] != 0

# Train data
df_0 = df[~is_test]
# Test data
df_1 = df[is_test]

X_cols_blacklist = [
    "id",
    "has_diabetes",
    "is_test",
]

# Genuine features
X_cols = [col for col in df.columns if col not in X_cols_blacklist]
# Target
y_col = "has_diabetes"

# Train data, feature part
X_0 = df_0[X_cols]
# Test data, feature part
X_1 = df_1[X_cols]
# Train data, target part
y_0 = df_0[y_col]
# Test data, target part
y_1 = df_1[y_col]

## Simple Logistic Regression Model

For the purpose of benchmarking, we select a logistic regression model that prognosticates the presence or absence of diabetes solely upon `"age"` and `"bmi"`.

To attain more accurate results, we shall apply a pipeline including oversampling and scaling techniques.

In [ ]:
X_cols_red = ["age", "bmi"]

# ColumnTransformer to select columns.
feature_selector = ColumnTransformer([("selector", "passthrough", X_cols_red)])

pipeline = Pipeline(
    [
        # Feature selection
        ("feature_selector", feature_selector),
        # Oversampler
        ("oversampler", RandomOverSampler(random_state=RSEED)),
        # Scaler for features (target scaling is not necessary)
        ("scaler", MinMaxScaler()),
        # Logistic regression
        ("model", LogisticRegression(random_state=RSEED)),
    ]
)

The following code cell fits the pipeline to the processed data and computes the parameters of the unscaled features within the linear regression model it contains.

In [ ]:
grid_search = GridSearchCV(
    pipeline,
    param_grid={},
    cv=10,
    scoring="accuracy",
    return_train_score=True,
)

grid_search.fit(X_0, y_0)

estimator = grid_search.best_estimator_

steps = estimator.named_steps

model = steps["model"]
# Coefficients and intercept are computed with respect
# to the scaled features.
coef_scaled = model.coef_[0]
intercept_scaled = model.intercept_[0]

scaler = steps["scaler"]
data_min = scaler.data_min_ 
data_max = scaler.data_max_ 
data_range = data_max - data_min 

# Back-transformation of coefficients and intercept to original scale
coef_orig = coef_scaled / data_range
intercept_orig = intercept_scaled - np.sum(coef_scaled * data_min / data_range)

print("Best parameters :")
print(f"  Coefficients : {coef_orig}")
print(f"  Intercept    : {intercept_orig}")

The coefficients for the features are positive, as one would naively expect: the older an individual, the more likely he is to suffer from diabetes; an analogous argument applies to BMI.

We proceed with the evaluation of the baseline model.

In [ ]:
z_1 = estimator.predict(X_1)

print("Classification Report:")
print(classification_report(y_1, z_1))

tpr = recall_score(y_1, z_1, pos_label=1)
print(f"True Positive Rate (Recall): {tpr:.2f}")

The resulting confusion matrix has the following values:

In [ ]:
cm = confusion_matrix(y_1, z_1)
sns.heatmap(cm, cmap="YlGnBu_r", annot=True, fmt=".0f")

plt.show()

To visualise the error, we plot all test data points coloured by their actual and predicted outcomes in the `"age"`–`"bmi"` plane. Moreover, we may include the decision boundary of the linear regression.

In [ ]:
# Scatterplot part

# Frame with plotting data
df_plot = X_1[["age", "bmi"]].copy()
df_plot["diagnosis"] = y_1
df_plot["prognosis"] = z_1
df_plot["outcome"] = df_plot.apply(classify_outcome, axis=1)

# Scatterplot
sns.scatterplot(
    data=df_plot,
    x="age",
    y="bmi",
    hue="outcome",
    palette=outcome_palette,
    s=20,
    edgecolor="k",
    alpha=0.7,
)

# Decision boundary part

# The decision boundary is described by 
# age_coeff * age + bmi_coeff * bmi + intercept = 0.
# We resolve this equation for bmi.

# Range of age
xx = np.array([df_plot["age"].min() - 1, df_plot["age"].max() + 1])

# Corresponding bmi values on the decision boundary line
yy = -(intercept_orig + coef_orig[0] * xx) / coef_orig[1]

# Plot decision boundary line
plt.plot(xx, yy, "k--", linewidth=2, label="Decision Boundary")

# Finalisation of the plot

plt.xlabel("Age (years)")
plt.ylabel("BMI (kg/sqm)")
plt.title("Classification Results with Decision Boundary")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## Decision Tree Model

As the Machine Learning model intended to optimise our target evaluation metric, we choose a decision tree classifier.

In [ ]:
X_cols_blacklist = ["dpf", "date"]
X_cols_red = [col for col in X_cols if col not in X_cols_blacklist]

feature_selector = ColumnTransformer([("selector", "passthrough", X_cols_red)])

pipeline = Pipeline(
    [
        # Feature selection
        ("feature_selector", feature_selector),
        # Oversampler
        ("oversampler", RandomOverSampler(random_state=RSEED)),
        # Scaling is not strictly necessary for Decision Trees
        # ("scaler", MinMaxScaler()),
        # Decision tree classifier
        ("model", DecisionTreeClassifier(random_state=RSEED)),
    ]
)

We shall fit the model to the training data. A parameter grid serves to identify a superior variant of the model.

In [ ]:
param_grid = {
    "model__max_depth": [2, 4, 8, 16, None],
    "model__min_samples_split": [2, 4, 8],
    "model__min_samples_leaf": [1, 2, 4, 8],
}

grid_search = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=5,
    scoring="recall",
    n_jobs=-1,
    verbose=1,
)

grid_search.fit(X_0, y_0)

print("Best parameters :")
pad = max([len(k) for k in param_grid]) + 1
for k,v in grid_search.best_params_.items():
    print(f"  {k:{pad}}: {v}")
print("Best CV score   :", grid_search.best_score_)

We shall print the evaluation report of the model.

In [ ]:
z_1 = grid_search.predict(X_1)

print("Classification Report:")
print(classification_report(y_1, z_1))

tpr = recall_score(y_1, z_1, pos_label=1)
print(f"True Positive Rate (Recall): {tpr:.2f}")

Once more, let us display the confusion matrix.

In [ ]:
cm = confusion_matrix(y_1, z_1)
sns.heatmap(cm, cmap="YlGnBu_r", annot=True, fmt=".0f")
plt.show()

To complete the parallelism with the error evaluation of the baseline model, we plot all test data points coloured by their actual and predicted outcomes in the `"age"`–`"bmi"` plane.

In [ ]:
# Frame with plotting data.
df_plot = X_1[["age", "bmi"]].copy()
df_plot["diagnosis"] = y_1
df_plot["prognosis"] = z_1
df_plot["outcome"] = df_plot.apply(classify_outcome, axis=1)

# Scatterplot
sns.scatterplot(
    data=df_plot,
    x="age",
    y="bmi",
    hue="outcome",
    palette=outcome_palette,
    s=20,
    edgecolor="k",
    alpha=0.7,
)

# Finalisation of the plot

plt.xlabel("Age (years)")
plt.ylabel("BMI (kg/sqm)")
plt.title("Classification Results with Decision Boundary")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

One may clearly observe that, in contrast to the baseline model, there is no longer a "decision boundary".